## should be useless, only use read_icon_model_3D.py !

In [1]:
%run ../confg

In [2]:
"""Read in the 3D ICON Model

functions used from outside:
- read_icon_fixed_point() need dask to read it in, a lot of RAM used
- read_icon_fixed_point_and_time()
"""

import math
from functools import partial

import metpy.calc as mpcalc
import numpy as np
import xarray as xr
from metpy.units import units

In [3]:
def read_icon_fixed_point(nearest_grid_cell, day=16, variant="ICON"):
    """
    Reads ICON 3D datasets for a given day and a given grid cell
    NOTE: Since the files are large we need dask to not get a overflow in RAM used

    Parameters:
    - nearest_grid_cell: The index of the nearest cell

    Returns:
    - Combined xarray dataset along dimensions, with selected ICON variables.
    """
    if day not in [15, 16]:
        raise ValueError("Only October day 15 or 16 is available!")

    # Preprocess function to select data at a specific location
    def _preprocess(x):
        return x.isel(ncells=nearest_grid_cell)

    # Use open_mfdataset with the partial function as a preprocess argument
    partial_func = partial(_preprocess)

    if variant == "ICON":  # put together read icon & read icon2TE script
        file_pattern = f'ICON_BLM-GUF_20171015T1200Z_CAP02_2D-3D_10min_1km_all_201710{str(day)}T????00Z.nc'
        folder = icon_folder_3D
    elif variant == "ICON2TE":
        file_pattern = f'ICON_2TE_BLM-GUF_20171015T1200Z_CAP02_2D-3D_10min_1km_all_201710{str(day)}T????00Z.nc'
        folder = icon2TE_folder_3D
    else:
        print("invalid model variant, either ICON or ICON2TE")

    # Load and concatenate datasets automatically by coordinates
    ds = xr.open_mfdataset(
        folder + f"/{file_pattern}",
        combine='by_coords',
        preprocess=partial_func
    )
    
    # Handling 'z_ifc' to include only in the first dataset
    if 'z_ifc' in ds.variables:
        z_ifc = ds['z_ifc'].isel(time=0).expand_dims('time')  # Get 'z_ifc' from the first time point
        ds = ds.drop_vars('z_ifc', errors='ignore')  # Drop 'z_ifc' from all datasets
        ds = ds.assign({'z_ifc': z_ifc})  # Reassign 'z_ifc' only for the first time point

    return ds

In [4]:
# 3 Mal ", dims=["height"] entfernt"
def convert_calc_variables(df):
    """
    Converts and calculates meteorological variables for a xarray Dataset.

    Parameters:
    - df: A xarray Dataset containing the columns 'p' for pressure in Pa
          and 'th' for potential temperature in Kelvin.

    Returns:
    - A xarray Dataset with the original data and new columns:
      'pressure' in hPa and 'temperature' in degrees Celsius.
    """

    # Convert pressure from Pa to hPa
    df['pressure'] = df['pres'] / 100.0
    p = df['pressure'].values * units.hPa

    # Calculate temperature from potential temperature
    temp_C = df["temp"] - 273.15
    df["temperature"] = temp_C

    # calculate specific + relativ humidity
    temp_values = temp_C.values * units.degC
    specific_humidity = df["qv"].values * 1000 * units("g/kg")
    df['specific_humidity'] = xr.DataArray(data=specific_humidity.magnitude)

    # the variables that go into mpcalc have to Arrays (Quantitys) without Dimension (important to take .values before)
    relative_humidity = mpcalc.relative_humidity_from_specific_humidity(p, temp_values, specific_humidity).to('percent')
    df['relative_humidity'] = xr.DataArray(data=relative_humidity.magnitude)

    # calculate dewpoint
    rh_values = df["relative_humidity"].values * units.percent
    dewpoint = mpcalc.dewpoint_from_relative_humidity(temp_values, rh_values).to("degC")
    df['dewpoint'] = xr.DataArray(data=dewpoint.magnitude)

    return df

In [5]:
def find_min_index(ds_icon, my_lon, my_lat):
    """
    Distances are relatively short where the curvature of the Earth can be neglected (fast 0.04 seconds)
    deleted old function, still in 2TE version
    """
    # Convert degrees to radians for calculation
    lon_rad = np.radians(my_lon)
    lat_rad = np.radians(my_lat)

    lon_diff_squared = (ds_icon.clon - lon_rad) ** 2
    lat_diff_squared = (ds_icon.clat - lat_rad) ** 2

    # Sum the squared differences to get squared Euclidean distances
    squared_distances = lon_diff_squared + lat_diff_squared

    # Find the index of the minimum squared distance
    min_idx = squared_distances.argmin()
    return min_idx.values

In [6]:
def read_icon_fixed_point_and_time(day, hour, my_lon, my_lat, variant = "ICON"):
    """
    Read Icon 3D model at a fixed point and a fixed time

    """
    
    if day not in [15, 16]:
        raise ValueError("Only October day 15 or 16 is available!")

    formatted_hour = f"{hour:02d}"

    if variant == "ICON":  # put together read icon & read icon2TE script
        icon_file = f'ICON_BLM-GUF_20171015T1200Z_CAP02_2D-3D_10min_1km_all_201710{day}T{formatted_hour}0000Z.nc' 
        folder = icon_folder_3D       
    elif variant == "ICON2TE":
        icon_file = f'ICON_2TE_BLM-GUF_20171015T1200Z_CAP02_2D-3D_10min_1km_all_201710{day}T{formatted_hour}0000Z.nc'
        folder = icon2TE_folder_3D
    else:
        print("invalid model variant, either ICON or ICON2TE") 

    ds_icon = xr.open_dataset(f"{folder}/" + icon_file)
    
    min_idx = find_min_index(ds_icon, my_lon, my_lat)

    nearest_data = ds_icon.isel(ncells=min_idx).isel(time=0)

    return convert_calc_variables(nearest_data)  # calculate temp, pressure

In [7]:
if __name__ == '__main__':
    icon = read_icon_fixed_point_and_time(day=16, hour=12, my_lon=11.4011756,
                                                 my_lat=47.266076, variant = "ICON")
    icon2TE = read_icon_fixed_point_and_time(day=16, hour=12, my_lon=11.4011756,
                                                 my_lat=47.266076, variant = "ICON2TE")


#     print(df_nearest.data_vars)
#     df = read_icon_lidar()

#     df = read_icon_lidar(lon=station_files_zamg["IAO"]["lon"], lat=station_files_zamg["IAO"]["lat"])
#     print(df)
#     d = read_icon3D_2(day=16)
#     print(d)
#     df = read_icon_fixed_point(d.load(), my_lon=station_files_zamg["IAO"]["lon"], my_lat=station_files_zamg["IAO"]["lat"])
#     print(df.load())
    #Example usage with specified longitude and latitude


In [8]:
icon.isel("height")

<xarray.Dataset> Size: 538MB
Dimensions:            (vertices: 3, ncells_2: 1344922, vertices_2: 4,
                        height: 90, bnds: 2, height_2: 91, height_3: 91,
                        height_4: 1, height_5: 1, depth: 9, depth_2: 8,
                        dim_0: 90)
Coordinates:
    clon               float32 4B 0.1989
    clat               float32 4B 0.825
    elon               (ncells_2) float32 5MB ...
    elat               (ncells_2) float32 5MB ...
  * height             (height) float64 720B 1.0 2.0 3.0 4.0 ... 88.0 89.0 90.0
  * height_2           (height_2) float64 728B 1.0 2.0 3.0 ... 89.0 90.0 91.0
  * height_3           (height_3) float64 728B 1.0 2.0 3.0 ... 89.0 90.0 91.0
  * height_4           (height_4) float64 8B 2.0
  * height_5           (height_5) float64 8B 10.0
  * depth              (depth) float64 72B 0.0 5.0 20.0 ... 4.86e+03 1.458e+04
  * depth_2            (depth_2) float64 64B 5.0 20.0 ... 4.86e+03 1.458e+04
    time               datetime64[ns] 8B 2017-10-16T12:00:00
Dimensions without coordinates: vertices, ncells_2, vertices_2, bnds, dim_0
Data variables: (12/67)
    clon_bnds          (vertices) float32 12B ...
    clat_bnds          (vertices) float32 12B ...
    elon_bnds          (ncells_2, vertices_2) float32 22MB ...
    elat_bnds          (ncells_2, vertices_2) float32 22MB ...
    height_bnds        (height, bnds) float64 1kB ...
    height_3_bnds      (height_3, bnds) float64 1kB ...
    ...                 ...
    snowfrac_lc        float32 4B ...
    pressure           (height) float32 360B 29.71 37.53 49.79 ... 955.9 956.8
    temperature        (height) float32 360B -60.83 -62.54 ... 22.39 22.76
    specific_humidity  (dim_0) float32 360B 0.002992 0.002923 ... 8.658 8.811
    relative_humidity  (dim_0) float32 360B 0.8402 1.295 2.07 ... 48.92 48.72
    dewpoint           (dim_0) float32 360B -91.64 -90.51 -89.12 ... 11.14 11.41
Attributes:
    CDI:                  Climate Data Interface version 1.8.3rc (http://mpim...
    Conventions:          CF-1.6
    number_of_grid_used:  1
    uuidOfHGrid:          655488b8-6e60-ac09-a653-9b1ce37a2b20
    uuidOfVGrid:          5210aca5-6684-c009-3731-0a31182a3180
    institution:          Max Planck Institute for Meteorology/Deutscher Wett...
    title:                ICON simulation
    source:               git@gitlab.dkrz.de:icon/icon-nwp.git@1638fcbef3269d...
    history:              /work/bb1096/b380910/models/icon/icon-nwp_2TE//bin/...
    references:           see MPIM/DWD publications
    comment:              Julian Quimbayo-Duarte (b380910) on l30537 (Linux 4...